# 1. Creating a simple ODE model: the reservoir model

In the [Getting Started](../getting_started/tutorial-1-how-to-run-a-model.ipynb) series, we learnt how to use in-built PyBaMM models. In this series, our focus will be on how to build PyBaMM models from scratch, from the very simple reservoir model all the way up to the Doyle-Fuller-Newman model. We strongly advise to familiarise yourself with the Getting Started notebooks, if you have not done so, before following this series.

We will start creating and solving the reservoir model. This model sits at the interface between equivalent circuit models and physics-based models (can be seen as an instance of either) and it is a very good starting point as it is composed only of ordinary differential equations (ODEs). The model is described by the following equations:



In this notebook we create and solve the following simple ODE model:
$$
\begin{align*}
\frac{\mathrm{d} x_\mathrm{n}}{\mathrm{d} t} &= -\frac{I(t)}{Q_\mathrm{n}}, \\
\frac{\mathrm{d} x_\mathrm{p}}{\mathrm{d} t} &= \frac{I(t)}{Q_\mathrm{p}}, \\
V(t) &= U_\mathrm{p}(x_\mathrm{p}) - U_\mathrm{n}(x_\mathrm{n}) - I(t)R, \\
x_\mathrm{n}(0) &= x_{\mathrm{n}0}, \\
x_\mathrm{p}(0) &= x_{\mathrm{p}0}, \\
\end{align*}
$$
where $x_n$ and $x_p$ are the dimensionless stochiometries of the negative and positive electrodes, $I(t)$ is the current, $Q_\mathrm{n}$ and $Q_\mathrm{p}$ are the capacities of the negative and positive electrodes, $U_\mathrm{p}(x_\mathrm{p})$ and $U_\mathrm{n}(x_\mathrm{n})$ are the open circuit potentials of the positive and negative electrodes, and $R$ is the internal resistance of the battery.

We begin by importing the PyBaMM library into this notebook, along with NumPy and Matplotlib, which we use for plotting:


In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


## Setting up the model

In this section we will set up our model. There will be multiple steps involved. First, we will define the variables and parameters of the model. Next, we will define the model itself. Finally, we will discuss events (conditions that can be used to stop the solver), which are optional but very useful in battery modelling.

### Variables
First we need to define the variables of the model, which are the quantities we want to solve for ([state variables](https://en.wikipedia.org/wiki/State-space_representation#State_variables)) and the quantities they depend on (independent variables). In this case, we only need to define two state variables: the stoichiometry of each electrode. The independent variable is time, but this variable is already built in PyBaMM so we do not need to define it. Voltage can be computed directly from the stoichiometries so it is not a state variable and we do not need to define it (we will say more on this when we define the model).

In PyBaMM a state variable can be defined using the `pybamm.Variable` class, which takes as argument the name of the variable as a string. In this case, we define the two stoichiometries as variables:

In [2]:
x_n = pybamm.Variable("Negative electrode stoichiometry")
x_p = pybamm.Variable("Positive electrode stoichiometry")

It is worth clarifying the difference between the Python object name (e.g. `x_n`) and the PyBaMM variable name (e.g. Negative electrode stochiometry). The former is used to call the latter within the script/notebook. The variable name is used to identify the variable in the expression tree (more on expression trees [later](#expression-trees)), which is very useful when debugging the code, so it is advisable to make the name as informative as possible.

### Parameters
Next we need to define the parameters of the model. [Parameters](https://en.wikipedia.org/wiki/Parameter) are known quantities that define the specific system we are modelling. For example, the electrode capacities are parameters of the reservoir model: they are known and they determine the specific battery we are modelling. In PyBaMM, parameters act as placeholders in the model and their specific values are passed at a later stage, when simulating the model.

PyBaMM parameters are defined very similarly to variables, but now using the class `pybamm.Parameter`. For example, we can define:

In [3]:
Q_n = pybamm.Parameter("Negative electrode capacity [A.h]")
Q_p = pybamm.Parameter("Positive electrode capacity [A.h]")
R = pybamm.Parameter("Electrode resistance [Ohm]")
x_n_0 = pybamm.Parameter("Initial negative electrode stochiometry")
x_p_0 = pybamm.Parameter("Initial positive electrode stochiometry")

Again, it is worth highlighting the difference between the Python object name (e.g `Q_n`) and the PyBaMM parameter name (e.g. Negative electrode capacity [A.h]). The names of the parameters need to match those in the keys of the parameter values dictionary, which are passed when simulating the model as you might recall from the [Tutorial 4 - Setting parameter values](../getting_started/tutorial-4-setting-parameter-values.ipynb) notebook.

Sometimes we need to define parameters that, even though they are known, they depend on variables of the model. For example, in the reservoir model we observe that the open-circuit potentials $U_n$ and $U_p$ depend on the stoichiometries. In this case, we need to use the `pybamm.FunctionParameter` class which in addition to the parameter name it takes as argument a dictionary with the inputs of the function parameter:

In [4]:
i = pybamm.FunctionParameter("Current function [A]", {"Time [s]": pybamm.t})
U_p = pybamm.FunctionParameter("Positive electrode OCV", {"x_p": x_p})
U_n = pybamm.FunctionParameter("Negative electrode OCV", {"x_n": x_n})

As we mentioned earlier, the time variable is already built in PyBaMM as `pybamm.t`.

### The model

Now that we have defined the parameters and variables for the reservoir model, we can define the model itself. In PyBaMM a model can be defined using the [`pybamm.BaseModel`](https://docs.pybamm.org/en/stable/source/api/models/base_models/base_model.html) class. PyBaMM models can be composed by differential and/or algebraic equations. Differential equations have the form $\frac{\mathrm{d} x}{\mathrm{d} t} = f(x, y, t)$, where $x$ and $y$ are state variables, $t$ is time, and $f(x, y, t)$ is an arbitrary function that will refer as the right-hand-side (rhs) of the differential equation. Algebraic equations have the form $g(x, y, t) = 0$. For example, let's consider the following arbitrary system:

$$
\begin{align*}
\frac{\mathrm{d} x}{\mathrm{d} t} &= f(x, y, t), \\
g(x, y, t) &= 0, \\
x(0) &= x_{0},
\end{align*}
$$
with an output variable (i.e. a variable that is computed directly from the state and indepedent variables) $z = h(x, y, t)$. 


We will use this example to illustrate the main attributes of the `BaseModel` class. These four main attributes are:
1. `rhs` - a Python dictionary of the right-hand-side term for the differential equations, where the key is the state variable we are solving for (e.g. $x$) and the value is the rhs function (e.g. $f(x, y, t)$). For our example, we would write `{x: f(x, y, t)}`.
2. `algebraic` - a Python dictionary of the algebraic equations. The key is still a state variable (just for indexing purposes, e.g $y$) and the value is the function that is equal to zero (e.g. $g(x, y, t)$). For our example, we would write `{y: g(x, y, t)}`. It is worth reiterating that this imposes $g(x, y, t) = 0$, not $y = g(x, y, t)$.
3. `initial_conditions` - a Python dictionary of the initial conditions. Again, the key is the variable we are imposing an initial condition for and the value is the initial value. Even though mathematically initial conditions are only needed for differential equations, PyBaMM also requires them for algebraic equations and the value is used as an initial guess for the solver. For our example, we would write `{x: x_0, y: y_0}` (where `y_0` would be a reasonable value for y at the initial state).
4. `variables` - a Python dictionary of the output variables. The key is a string with the variable name (as a string) and the value is the expression for that variable. For our example, we would write `{"z": h(x, y, t)}`.

Now we have all the pieces we need to define the reservoir model. We start by initialising the model:

In [5]:
model = pybamm.BaseModel("reservoir model")

Note that we can pass the name of the model as a string when initialising it. This will be used by PyBaMM's plotting functionality and comes handy when plotting various models together. Next we define the rhs and initial conditions for our model:

In [6]:
model.rhs[x_n] = -i / (Q_n * 3600)
model.initial_conditions[x_n] = x_n_0
model.rhs[x_p] = i / (Q_p * 3600)
model.initial_conditions[x_p] = x_p_0

Note the factor of 3600 in the equation to convert the Ah to C.

Finally, we define the output variables for our model. Note that these can be any variables we might need to use (e.g. to analyse, plot...) after solving the model, including the state variables. For the reservoir model we define the stoichiometry of each electrode and the voltage:

In [7]:
model.variables = {
    "Negative electrode stoichiometry": x_n,
    "Positive electrode stoichiometry": x_p,
    "Voltage [V]": U_p - U_n - i * R,
}

The naming convention of the variables is up to the user, but we suggest sticking to the PyBaMM convention of the name followed by the units in square brackets. The only constraint is that state variables need to be given the same name as when they were defined.

### PyBaMM expressions

It is worth pausing here and discussing the concept of an "expression" in PyBaMM. Notice that in the model definition we have used expressions like `-i / Q_n` and `U_p - U_n -  i * R`. These expressions do not evaluate to a single value like similar expressions involving Python variables would. Instead, they are symbolic expressions that represent the mathematical equation.

The fundamental building blocks of a PyBaMM model are the expressions, either those for the RHS rate equations, the algebraic equations of the expression for the output variables such as $V(t)$. PyBaMM encodes each of these expressions using an "expression tree", which is a tree of operations (e.g. addition, multiplication, etc.), variables (e.g. $x_\mathrm{n}$, $x_\mathrm{p}$, $I(t)$, etc.), and functions (e.g. $\exp$, $\sin$, etc.), which can be evaluated at any point in time.

As an example, lets consider the expression for $\frac{\mathrm{d} x_\mathrm{n}}{\mathrm{d} t}$, given by `-i / Q_\mathrm{n}`. We can visualise the expression tree for this expression using the `render` method:

In [8]:
model.rhs[x_n].render()

/
├── -
│   └── Current function [A]
│       └── time
└── *
    ├── 3600.0
    └── Negative electrode capacity [A.h]


or print it to a .png file (which renders it more nicely) by typing: `model.rhs[x_n].visualise("x_n_rhs.png")`. You can also print the expression as a string using the `print` method:

In [9]:
print(model.rhs[x_n])

-Current function [A] / (3600.0 * Negative electrode capacity [A.h])


In any case, you can see that the expression tree is a symbolic representation of the mathematical equation, which can then be later on used by the PyBaMM solvers to solve the model equations over time.

### Events

An event is a condition that can be used to stop the solver, for example when the voltage reaches a certain value. Even though they are not a fundamental part of the model (and they are definitely optional), events are very useful to prevent the model reaching unfeasible regions (e.g. negative concentrations). In PyBaMM an event can be defined using the [`pybamm.Event`](https://docs.pybamm.org/en/stable/source/api/events/event.html) class. The class takes two required arguments: a string with the name of the event (for identification purposes) and an expression that is the condition to trigger the event. For example, let's consider the following event for the maximum stoichiometry in the negative electrode:

In [10]:
max_stoichiometry = pybamm.Event("Maximum negative stochiometry", 1 - x_n)

The expression `1 - x_n` is the condition that the event is looking for, and the solver will stop when this expression becomes zero. Note that the
expression must evaluate to a non-negative number for the duration of the simulation, and then become zero when the event is triggered.

PyBaMM models have an `events` attribute, which is of all the events of the model. When one of these events is triggered the solver will stop. For example, in the reservoir model we want the stoichiometries to be between 0 and 1, so we define:

In [11]:
model.events = [
    pybamm.Event("Minimum negative stochiometry", x_n),
    pybamm.Event("Maximum negative stochiometry", 1 - x_n),
    pybamm.Event("Minimum positive stochiometry", x_p),
    pybamm.Event("Maximum positive stochiometry", 1 - x_p),
]

## Using the model

Now that we have defined the model, we can proceed to solve it. However, before actually solving it, we need to define the parameter values we will use. 

### Parameter values
You should already be familiar with the [`pybamm.ParameterValues`](https://docs.pybamm.org/en/stable/source/api/parameters/parameter_values.html)
class, which is used to define the values of the parameters in the model (see [Tutorial 4 - Setting parameter values](../getting_started/tutorial-4-setting-parameter-values.ipynb) for a full recap). Recall that the parameter values are passed to the `pybamm.ParameterValues` class as a dictionary, where the key is the parameter name (i.e. the string we used when defining the parameter) and the value is the parameter value. For parameter functions, we can define them either as a normal or a lambda function. For the reservoir model, we define the following parameter values:

In [12]:
def graphite_LGM50_ocp_Chen2020(sto):
    u_eq = (
        1.9793 * np.exp(-39.3631 * sto)
        + 0.2482
        - 0.0909 * np.tanh(29.8538 * (sto - 0.1234))
        - 0.04478 * np.tanh(14.9159 * (sto - 0.2769))
        - 0.0205 * np.tanh(30.4444 * (sto - 0.6103))
    )

    return u_eq


def nmc_LGM50_ocp_Chen2020(sto):
    u_eq = (
        -0.8090 * sto
        + 4.4875
        - 0.0428 * np.tanh(18.5138 * (sto - 0.5542))
        - 17.7326 * np.tanh(15.7890 * (sto - 0.3117))
        + 17.5842 * np.tanh(15.9308 * (sto - 0.3120))
    )

    return u_eq


parameter_values = pybamm.ParameterValues(
    {
        "Current function [A]": lambda t: 1 + 0.5 * pybamm.sin(t / 100),
        "Initial negative electrode stochiometry": 0.9,
        "Initial positive electrode stochiometry": 0.3,
        "Negative electrode capacity [A.h]": 1.2,
        "Positive electrode capacity [A.h]": 1,
        "Electrode resistance [Ohm]": 0.1,
        "Positive electrode OCV": nmc_LGM50_ocp_Chen2020,
        "Negative electrode OCV": graphite_LGM50_ocp_Chen2020,
    }
)

### Solving the model

Finally we can proceed to solve the model. For this simple case, we can use the `pybamm.Simulation` class directly. We define the simulation by passing the model and the parameter values and then solve it for a given time interval (in this case 1 second):

In [13]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sol = sim.solve([0, 3600])

We can plot the solution by using the PyBaMM's standard plotting capabilities. In this case, as it is a custom model, we need to specify which variables we want to plot:

In [14]:
sol.plot(
    [
        "Voltage [V]",
        "Negative electrode stoichiometry",
        "Positive electrode stoichiometry",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=2519.87471894461, step=25.1987471894461), Ou…

In this tutorial we have introduced the main concepts to define a PyBaMM model and used them to build the reservoir model. In the [next notebook](./2-a-pde-model.ipynb) we will build on what we have learn to define a simple partial differential equation (PDE) model: the Single Particle Model for a half cell.

## References

The relevant papers for this notebook are:

In [15]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[3] Valentin Sulzer, Scott G. Marquis, Robert Timms, Martin Robinson, and S. Jon Chapman. Python Battery Mathematical Modelling (PyBaMM). Journal of Open Research Software, 9(1):14, 2021. doi:10.5334/jors.309.

